# Genetic Algorithm for combinatorial optimisation

This notebook showcases how to use some of the building blocks provided by EvoLP to solve a combinatorial problem.

In [1]:
using Statistics
using EvoLP
using OrderedCollections

In this example,  we will solve the 8-queen problem.
This is a constraint satisfaction problem in which the goal is to place 8 queens in a chess board such that neither of them _check_ each other.

![One queen on the board](8-queens01.png)

In the figure above, we have placed a queen represented by a blue dot. All conflicting cells have been highlighted. The problem becomes harder when we add more queens to the board:

![2 queens on the board](8-queens02.png)

We can solve this problem using a Genetic Algorithm (GA) that deals with the constraints.

## Implementing the solution

To implement the solution, we need to deal with the constraints in some way.

In a GA, we use vectors as individuals.
In this case, our individuals are **permutations** of the numbers 1 to 8

In [3]:
@doc rand_pop_int_perm

```
rand_pop_int_perm(n, d, pool)
```

Generate a population of `n` permutation vector individuals, of size `d` and with values sampled from `pool`.

Usually `d` would be equal to `length(pool)`.


In [4]:
pop_size = 100
population = rand_pop_int_perm(pop_size, 8, 1:8)
first(population, 3)

3-element Vector{Vector{Int64}}:
 [4, 6, 2, 5, 8, 1, 3, 7]
 [2, 4, 7, 3, 1, 6, 8, 5]
 [6, 7, 8, 1, 5, 4, 3, 2]

In a GA, we have selection, crossover and mutation.

However, since we're dealing with permutations, we are restricted to use some specific operators that do not violate our constraints:

In [5]:
@doc TournamentSelectionSteady

Tournament parent selection with tournament size `k`.


In [6]:
@doc OrderOneCrossover

Order 1 crossover (OX1) for permutation-based individuals.


In [7]:
@doc SwapMutation

Swap mutation for permutation-based individuals.


In [8]:
S = TournamentSelectionSteady(5);
C = OrderOneCrossover();
M = SwapMutation();

We can use the `Logbook` to record statistics about our run:

In [9]:
statnames = ["mean_eval", "max_f", "min_f", "median_f"]
fns = [mean, maximum, minimum, median]
thedict = LittleDict(statnames, fns)
thelogger = Logbook(thedict)

Logbook(LittleDict{AbstractString, Function, Vector{AbstractString}, Vector{Function}}("mean_eval" => Statistics.mean, "max_f" => maximum, "min_f" => minimum, "median_f" => Statistics.median), NamedTuple{(:mean_eval, :max_f, :min_f, :median_f)}[])

And now we are ready to use all our building blocks to construct our algorithm:

In [10]:
function mySteadyGA(logbook, f, pop, k_max, S, C, M, mrate)
    n = length(pop)
    for _ in 1:k_max
        fitnesses = f.(pop)
        parents = select(S, fitnesses)  # this will return 2 parents
        parents = vcat(parents, select(S, fitnesses))  # add two more parents
        offspring = [cross(C, pop[parents[1]], pop[parents[2]])]  # get first kid
        offspring = vcat(offspring, [cross(C, pop[parents[3]], pop[parents[4]])])  # get 2nd
        pop = vcat(pop, offspring)
        # mutation loop
        for i in eachindex(pop)
            if rand() <= mrate
                pop[i] = mutate(M, pop[i])
            end
        end
        fitnesses = f.(pop)

        compute!(logbook, fitnesses)
        worst1 = argmax(fitnesses)
        deleteat!(pop, worst1)
        deleteat!(fitnesses, worst1)
        worst2 = argmax(fitnesses)
        deleteat!(pop, worst2)
        deleteat!(fitnesses, worst2)
    end

    best, best_i = findmin(f, pop)
    n_evals = 2 * k_max * n + n
    result = Result(best, pop[best_i], pop, k_max, n_evals)
    return result
end

mySteadyGA (generic function with 1 method)

In [11]:
# now we do sum of constraints as the function
function check_constraints(x)
    # row constraint
    # no row constraint due to permutation phenotype
    # column constraint
    # no column constraint due to single-dimension
    # cross constraint
    # [i, j]
    # [i-1, j-1] = top-left diag
    # [i-1, j+1] = top-right diag
    # [i+1, j-1] = bottom-left diag
    # [i+1, j+1] = bottom-right diag

    # rows are values in x
    # columns are indices from 1:8
    fitness = []
    for q in 1:8
        tl = collect(zip(x[q]:-1:1, q:-1:1))
        tr = collect(zip(x[q]:-1:1, q:1:8))
        bl = collect(zip(x[q]:1:8, q:-1:1))
        br = collect(zip(x[q]:1:8, q:1:8))

        constraints = Set(vcat(tl, tr, bl, br))
        delete!(constraints, (x[q], q))
        q_fit = sum([(i, j) in constraints ? 1 : 0 for (i, j) in zip(x, 1:8)])
        push!(fitness, q_fit)
    end

    return sum(fitness)
end

check_constraints (generic function with 1 method)

In [14]:
result  = mySteadyGA(thelogger, check_constraints, population, 10000, S, C, M, 0.8);

In [15]:
@show optimum(result)
@show optimizer(result)
@show f_calls(result)
@show thelogger.records[end]

optimum(result) = 0
optimizer(result) = Any[6, 3, 7, 2, 8, 5, 1, 4]
f_calls(result) = 2000100
thelogger.records[end] = (mean_eval = 9.92156862745098, max_f = 26, min_f = 0, median_f = 10.0)


(mean_eval = 9.92156862745098, max_f = 26, min_f = 0, median_f = 10.0)

In [109]:
function drawboard(x)
    b = fill("◻",8,8)
    for i in 1:2:8
        b[i,2:2:8] .= "◼"
    end
    for i in 2:2:8
        b[i, 1:2:8] .= "◼"
    end
    for (i,j) in zip(x,1:8)
        b[i, j] = "♛"
    end
    for i in 1:8
        println(join(b[i,:]))
    end
end

drawboard (generic function with 1 method)

In [110]:
drawboard(optimizer(result))

◻◼◻◼◻◼♛◼
◼◻◼♛◼◻◼◻
◻♛◻◼◻◼◻◼
◼◻◼◻◼◻◼♛
◻◼◻◼◻♛◻◼
♛◻◼◻◼◻◼◻
◻◼♛◼◻◼◻◼
◼◻◼◻♛◻◼◻
